In [1]:
%matplotlib inline
from nfl.fdb import parser as nfl
from nfl.fdb.const import *
from nfl import storage as nfl_db
YEARS = list(range(1995, 2012))

## Mongo DB commands
mongod --dbpath /Users/eczech/data/dbs/mongodb/data

In [10]:
import imp
imp.reload(nfl)
imp.reload(nfl_db)

<module 'nfl.storage' from '/Users/eczech/repos/portfolio/functional/sports/python/nfl/storage.py'>

## Get Teams

In [15]:
for year in YEARS:
    print('Collecting team data for year {}'.format(year))
    teams = nfl.get_teams(year)
    nfl_db.insert_data(DB, COLL_TEAMS, teams, ['Year', 'Id'])

In [17]:
teams = nfl_db.get_data(DB, COLL_TEAMS)
teams.head()

,Id,Link,Name,Year
0,miami-dolphins,/teams/nfl/miami-dolphins,Miami Dolphins,2012
1,new-england-patriots,/teams/nfl/new-england-patriots,New England Patriots,2012
2,new-york-jets,/teams/nfl/new-york-jets,New York Jets,2012
3,baltimore-ravens,/teams/nfl/baltimore-ravens,Baltimore Ravens,2012
4,cincinnati-bengals,/teams/nfl/cincinnati-bengals,Cincinnati Bengals,2012


## Get Rosters

In [18]:
for year in YEARS:
    print('Collecting roster data for year {}'.format(year))
    teams = nfl_db.get_data(DB, COLL_TEAMS, {'Year': year})
    for team_id in teams['Id']:
        try:
            roster = nfl.get_roster(team_id, year)
        except:
            print('Failed to find roster for {} {}'.format(year, team_id))
            continue
        roster['TeamId'] = team_id
        nfl_db.insert_data(DB, COLL_ROSTR, roster, ['Year', 'TeamId', 'PlayerLink'])

In [19]:
rosters = nfl_db.get_data(DB, COLL_ROSTR)
rosters.head()

[Mongo] On extraction of document #10000
[Mongo] On extraction of document #20000
[Mongo] On extraction of document #30000


,Birthdate,College,G,GS,Number,Player,PlayerLink,Pos,TeamId,Year
0,1988-09-06 00:00:00,Texas,16,16,94,Sam Acho,/players/sam-acho-achosa01,DE,arizona-cardinals,2012
1,1985-06-17 00:00:00,Louisiana-Lafayette,15,0,27,Michael Adams,/players/michael-adams-adamsmi04,DB,arizona-cardinals,2012
2,1981-03-29 00:00:00,Louisiana-Lafayette,15,10,74,D'Anthony Batiste,/players/danthony-batiste-batisda02,OT,arizona-cardinals,2012
3,1990-06-17 00:00:00,Presbyterian,16,0,31,Justin Bethel,/players/justin-bethel-betheju01,DB,arizona-cardinals,2012
4,1983-11-02 00:00:00,Nebraska,16,0,55,Stewart Bradley,/players/stewart-bradley-bradlst02,LB,arizona-cardinals,2012


## Get Games

In [20]:
for year in YEARS:
    print('Collecting game data for year {}'.format(year))
    teams = nfl_db.get_data(DB, COLL_TEAMS, {"Year": year})
    for team_id in teams['Id']:
        try:
            games = nfl.get_games(team_id, year)
        except:
            print('Failed to find game data for {} {}'.format(year, team_id))
            continue
        games['TeamId'] = team_id
        nfl_db.insert_data(DB, COLL_GAMES, games, ['Year', 'TeamId', 'Date'])


In [21]:
games = nfl_db.get_data(DB, COLL_GAMES)
games.head()

[Mongo] On extraction of document #10000


,Attend,Date,GameType,Location,Opponent,OpponentLink,OutcomeLink,Result,TeamId,Year
0,"60,032",2012-09-09,Regular Season,"University of Phoenix Stadium (Glendale, AZ)",Seattle,/teams/nfl/seattle-seahawks/results/2012,/games/boxscore.html?gid=2012090911,W 20-16,arizona-cardinals,2012
1,"68,756",2012-09-16,Regular Season,"Gillette Stadium (Foxborough, MA)",@ New England,/teams/nfl/new-england-patriots/results/2012,/games/boxscore.html?gid=2012091605,W 20-18,arizona-cardinals,2012
2,"60,436",2012-09-23,Regular Season,"University of Phoenix Stadium (Glendale, AZ)",Philadelphia,/teams/nfl/philadelphia-eagles/results/2012,/games/boxscore.html?gid=2012092311,W 27-6,arizona-cardinals,2012
3,"60,183",2012-09-30,Regular Season,"University of Phoenix Stadium (Glendale, AZ)",Miami,/teams/nfl/miami-dolphins/results/2012,/games/boxscore.html?gid=2012093009,"W 24-21, OT",arizona-cardinals,2012
4,"54,653",2012-10-04,Regular Season,"Edward Jones Dome (St. Louis, MO)",@ St. Louis,/teams/nfl/st-louis-rams/results/2012,/games/boxscore.html?gid=2012100401,L 17-3,arizona-cardinals,2012


## Get Game Stats

In [22]:
for year in YEARS:
    print('Collecting game statistic data for year {}'.format(year))
    games = nfl_db.get_data(DB, COLL_GAMES, {"Year": year})
    for i, r in games[['GameType', 'OutcomeLink']].drop_duplicates().iterrows():
        game_type = r['GameType']
        assert game_type, 'Game type not present for row "{}"'.format(r)
        if game_type not in ['Regular Season', 'Preseason', 'Postseason']:
            raise ValueError('Found unexpected game type {} in row "{}"'.format(game_type, r))
        if game_type == 'Preseason':
            continue
        box_score_link = r['OutcomeLink']
        if pd.isnull(box_score_link):
            print('Ignoring null box score link for row "{}"'.format(r))
            continue
        
        try:
            stats = nfl.get_game_stats(box_score_link, year)
        except:
            print('Failed to find game statistic data for {}, box score = {}'.format(year, box_score_link))
            continue
        for stat_cat in stats:
            # Ignore team level statistics
            if stat_cat == 'Team Stats':
                continue
            col_name = stat_cat.replace(' ', '_').lower()
            col_name = COLL_STATS + '_' + col_name
            stat_data = stats[stat_cat]
            if len(stat_data) == 0:
                continue
            stat_data['OutcomeLink'] = box_score_link
            stat_data = stat_data.fillna('')
            nfl_db.insert_data(DB, col_name, stat_data, ['Year', 'OutcomeLink', 'RowLink', 'RowName'])


Ignoring null box score link for row "GameType       Regular Season
OutcomeLink               NaN
Name: 237, dtype: object"


In [169]:
nfl_db.get_collections(DB)

['fdb_games',
 'fdb_roster',
 'fdb_stats_defense',
 'fdb_stats_fumbles',
 'fdb_stats_kicking',
 'fdb_stats_kickoff_returns',
 'fdb_stats_kickoffs',
 'fdb_stats_passing',
 'fdb_stats_punt_returns',
 'fdb_stats_punting',
 'fdb_stats_receiving',
 'fdb_stats_rushing',
 'fdb_stats_team_stats',
 'fdb_teams',
 'system.indexes']

In [170]:
stats = nfl_db.get_data(DB, 'fdb_stats_defense')
stats.head()

,Ast,Avg,Int,Lg,OutcomeLink,RowLink,RowName,Sack,Solo,TD,Tot,Yds,YdsL,Year
0,4,0.00,0,0,/games/boxscore.html?gid=2012090911,/players/kj-wright-wrighkj01,K.J. Wright,0.0,5,0,9,0,0.0,2012
1,0,0.00,0,0,/games/boxscore.html?gid=2012090911,/players/kam-chancellor-chancka01,Kam Chancellor,0.0,6,0,6,0,0.0,2012
2,0,0.00,1,0,/games/boxscore.html?gid=2012090911,/players/richard-sherman-shermri01,Richard Sherman,0.0,4,0,4,0,0.0,2012
3,2,0.00,0,0,/games/boxscore.html?gid=2012090911,/players/bobby-wagner-wagnebo01,Bobby Wagner,0.0,2,0,4,0,0.0,2012
4,2,0.00,0,0,/games/boxscore.html?gid=2012090911,/players/brandon-browner-brownbr01,Brandon Browner,0.0,2,0,4,0,0.0,2012


## Scratch

In [72]:
for k in stats:
    print(k, stats[k]['RowLink'].unique(), len(stats[k]))

Defense ['/players/earl-thomas-thomaea03' '/players/bobby-wagner-wagnebo01'
 '/players/kam-chancellor-chancka01' '/players/tony-mcdaniel-mcdanto01'
 '/players/kj-wright-wrighkj01' '/players/malcolm-smith-smithma15'
 '/players/richard-sherman-shermri01' '/players/walter-thurmond-thurmwa01'
 '/players/brandon-mebane-mebanbr01' '/players/benson-mayowa-mayowbe01'
 '/players/obrien-schofield-schofob01' '/players/red-bryant-bryanre01'
 '/players/heath-farwell-farwehe01' '/players/russell-wilson-wilsoru01'
 '/players/byron-maxwell-maxweby01' '/players/chris-maragos-maragch01'
 '/players/danthony-smith-smithda19' '/players/thomas-davis-davisth02'
 '/players/luke-kuechly-kuechlu01' '/players/charles-godfrey-godfrch02'
 '/players/jon-beason-beasojo01' '/players/josh-thomas-thomajo12'
 '/players/captain-munnerlyn-munneca01' '/players/star-lotulelei-lotulst01'
 '/players/greg-hardy-hardygr01' '/players/mario-addison-addisma01'
 '/players/dj-moore-mooredj01' '/players/jj-jansen-jansejj01'
 '/player

In [10]:
stats['Kicking']

,RowLink,RowName,PAT,FG,0-19,20-29,30-39,40-49,50+,Lg,Miss,Pts,Year
0,/players/steven-hauschka-hauscst01,Steven Hauschka,0/0,2/2,0/0,1/1,0/0,1/1,0/0,40,--,6,2013
1,/players/graham-gano-ganogr01,Graham Gano,1/1,0/0,0/0,0/0,0/0,0/0,0/0,0,--,1,2013
